In [ ]:
%load_ext sql

In [ ]:
!pip install ipython-sql==0.4.1

In [ ]:
!pip install SQLAlchemy==1.4.49

In [ ]:
%sql postgresql://*****@kor-financial-market-workgroup.851725544098.ap-northeast-2.redshift-serverless.amazonaws.com:5439/dev

# 스키마 생성

In [ ]:
%%sql

CREATE SCHEMA raw_data;

# 테이블 생성

In [ ]:
%%sql

CREATE TABLE raw_data.fce_by_all_purpose (
    STAT_CODE VARCHAR(10),
    STAT_NAME VARCHAR(255),
    ITEM_CODE1 INT,
    ITEM_NAME1 VARCHAR(255),
    ITEM_CODE2 INT,
    ITEM_NAME2 VARCHAR(255),
    ITEM_CODE3 INT,
    ITEM_NAME3 VARCHAR(255),
    ITEM_CODE4 INT,
    ITEM_NAME4 VARCHAR(255),
    UNIT_NAME VARCHAR(50),
    WGT FLOAT,
    TIME INT,
    DATA_VALUE FLOAT
);

In [ ]:
%%sql

DROP TABLE IF EXISTS public.won_us_trading_base_rate;
CREATE TABLE raw_data.won_us_trading_base_rate (
    STAT_CODE VARCHAR(10),
    STAT_NAME VARCHAR(100),
    ITEM_CODE1 INT,
    ITEM_NAME1 VARCHAR(100),
    ITEM_CODE2 FLOAT,
    ITEM_NAME2 FLOAT,
    ITEM_CODE3 FLOAT,
    ITEM_NAME3 FLOAT,
    ITEM_CODE4 FLOAT,
    ITEM_NAME4 FLOAT,
    UNIT_NAME VARCHAR(10),
    WGT FLOAT,
    TIME INT,
    DATA_VALUE FLOAT
);

In [ ]:
%%sql

DROP TABLE IF EXISTS public.consumer_price_index;
CREATE TABLE raw_data.consumer_price_index (
    STAT_CODE VARCHAR(20),
    STAT_NAME VARCHAR(255),
    ITEM_CODE1 VARCHAR(20),
    ITEM_NAME1 VARCHAR(255),
    ITEM_CODE2 VARCHAR(20),
    ITEM_NAME2 VARCHAR(255),
    ITEM_CODE3 VARCHAR(20),
    ITEM_NAME3 VARCHAR(255),
    ITEM_CODE4 VARCHAR(20),
    ITEM_NAME4 VARCHAR(255),
    UNIT_NAME VARCHAR(50),
    WGT NUMERIC,
    TIME INTEGER,
    DATA_VALUE NUMERIC
);


# 테이블 스키마 확인

In [ ]:
%%sql

select * from pg_namespace;

In [ ]:
%%sql

select * from pg_tables where schemaname = 'raw_data' order by tablename;

In [ ]:
%%sql

SELECT * FROM information_schema.columns WHERE table_name LIKE '%fce%';

In [ ]:
%%sql

SELECT * FROM information_schema.columns WHERE table_name LIKE '%consumer%';

In [ ]:
%%sql

SELECT * FROM information_schema.columns WHERE table_name LIKE '%won%';

# S3 to Redshift 적재

In [ ]:
%%sql

copy raw_data.fce_by_all_purpose
from 's3://kor-financial-market/final_consumption_expenditure/fce_by_all_purpose.csv'
credentials 'aws_iam_role=arn:aws:iam::851725544098:role/redshift.read.s3'
delimiter ',' dateformat 'auto' timeformat 'auto' IGNOREHEADER 1 removequotes;

In [ ]:
%%sql

copy raw_data.won_us_trading_base_rate
from 's3://kor-financial-market/final_consumption_expenditure/won_us_trading_base_rate.csv'
credentials 'aws_iam_role=arn:aws:iam::851725544098:role/redshift.read.s3'
delimiter ',' dateformat 'auto' timeformat 'auto' IGNOREHEADER 1 removequotes;

In [ ]:
%%sql

copy raw_data.consumer_price_index
from 's3://kor-financial-market/final_consumption_expenditure/consumer_price_index.csv'
credentials 'aws_iam_role=arn:aws:iam::851725544098:role/redshift.read.s3'
delimiter ',' dateformat 'auto' timeformat 'auto' IGNOREHEADER 1 removequotes;

In [ ]:
%%sql

SELECT * FROM sys_load_error_detail order by start_time desc;

In [ ]:
%%sql

select * from raw_data.won_us_trading_base_rate order by TIME desc limit 10;

# 가공 테이블 생성

In [ ]:
%%sql

ALTER TABLE raw_data.won_us_trading_base_rate ADD COLUMN charge_value NUMERIC;

In [ ]:
%%sql

WITH cte AS (
    SELECT
        *,
        LAG(DATA_VALUE) OVER (ORDER BY TIME) AS prev_data_value
    FROM
        raw_data.won_us_trading_base_rate
)
UPDATE raw_data.won_us_trading_base_rate
SET charge_value = COALESCE(cte.DATA_VALUE - cte.prev_data_value, 0)
FROM cte
WHERE raw_data.won_us_trading_base_rate.TIME = cte.TIME
  AND raw_data.won_us_trading_base_rate.STAT_CODE = cte.STAT_CODE;

In [ ]:
%%sql

CREATE TABLE analytics_data.correlation_results AS
WITH combined_data AS (
    SELECT
        a.ITEM_NAME1 AS item1,
        b.ITEM_NAME1 AS item2,
        a.TIME,
        a.DATA_VALUE AS value1,
        b.DATA_VALUE AS value2
    FROM
        raw_data.fce_by_all_purpose a
    JOIN
        raw_data.consumer_price_index b
    ON
        a.TIME = b.TIME
)
SELECT
    item1,
    item2,
    CORR(value1, value2) AS correlation_coefficient
FROM
    combined_data
GROUP BY
    item1, item2
ORDER BY
    item1, item2;
